In [1]:
import geopandas as gpd
import pandas as pd
import numpy as np
import glob
import rasterio
from shapely.geometry import Polygon
from rasterio.mask import mask
from rasterio.merge import merge

/home/jupyter-peta_admin/.local/lib/python3.8/site-packages/geopandas/_compat.py:112: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [2]:
from google.cloud import storage

client = storage.Client()
ls_=list(client.list_blobs('bps-gcp-bucket', prefix='Degree of Urbanisation/GHSL data/2010/zip/'))

In [3]:
storage_client = storage.Client()
for i in ls_:
    data_=str(i).split('/')[4].split(',')[0]
    if len(data_)>3:
        bucket = storage_client.get_bucket('bps-gcp-bucket')
        source_blob_name='Degree of Urbanisation/GHSL data/2010/zip/'+data_
        blob = bucket.blob(source_blob_name)
        blob.download_to_filename('data/input/GHSL Data/2010/zip/'+data_)

In [4]:
import zipfile
data_=glob.glob('data/input/GHSL Data/2010/zip/*.zip')
for i in data_:
    with zipfile.ZipFile(i, 'r') as zip_ref:
        zip_ref.extractall('data/input/GHSL Data/2010/tif')

In [6]:
data_prov=gpd.read_file('data/input/Master_Data/Prov_20201_ByIndo(1).shp').to_crs('ESRI:54009')[['geometry','kdprov']]

In [7]:
ls_j=glob.glob('data/input/GHSL Data/2010/tif/*.tif')
for j in ls_j:
    with rasterio.open(j) as src:
        xmin=src.bounds.left
        xmax=src.bounds.right
        ymin=src.bounds.bottom
        ymax=src.bounds.top
        gpd_box=gpd.GeoDataFrame({'geometry':[Polygon([(xmin,ymin), (xmax, ymin), (xmax, ymax), (xmin, ymax)])]},crs='ESRI:54009')
        y_=gpd.sjoin(gpd_box,data_prov,op='intersects')['kdprov'].tolist()
        name_=j.split('/')[-1]
        for y1_ in y_:
            shapes=data_prov.query('kdprov==@y1_').geometry
            out_image, out_transform = mask(src, shapes, crop=True)
            out_meta = src.meta
            out_meta.update({"driver": "GTiff",
                 "height": out_image.shape[1],
                 "width": out_image.shape[2],
                 "transform": out_transform})
            with rasterio.open("data/input/GHSL Data/2010/tif_prov/2010_"+y1_+"_"+name_, "w", **out_meta) as dest:
                dest.write(out_image)

/opt/tljh/user/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3338: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):
/opt/tljh/user/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3338: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):
/opt/tljh/user/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3338: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):
/opt/tljh/user/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3338: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predic

In [8]:
for i in data_prov.kdprov:
    y_=glob.glob('data/input/GHSL Data/2010/tif_prov/2010_'+i+'*')
    dest, output_transform=merge(y_)
    with rasterio.open(y_[0]) as src:
        out_meta = src.meta.copy()    
    out_meta.update({"driver": "GTiff",
                 "height": dest.shape[1],
                 "width": dest.shape[2],
                 "transform": output_transform})
    with rasterio.open("data/input/GHSL Data/2010/prov/2010_"+i+".tif", "w", **out_meta) as dest1:
        dest1.write(dest)

In [9]:
def upload_blob(bucket_name, source_file_name, destination_blob_name):
    """Uploads a file to the bucket."""
    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)

    blob.upload_from_filename(source_file_name)

    print(
        f"File {source_file_name} uploaded to {destination_blob_name}."
    )

upload_2010=glob.glob('data/input/GHSL Data/2010/prov/2010_*')
for i in upload_2010:
    y_=i.split('/')[-1]
    upload_blob('bps-gcp-bucket',i,'Degree of Urbanisation/GHSL data/2010/prov/'+y_)

File data/input/GHSL Data/2010/prov/2010_76.tif uploaded to Degree of Urbanisation/GHSL data/2010/prov/2010_76.tif.
File data/input/GHSL Data/2010/prov/2010_33.tif uploaded to Degree of Urbanisation/GHSL data/2010/prov/2010_33.tif.
File data/input/GHSL Data/2010/prov/2010_52.tif uploaded to Degree of Urbanisation/GHSL data/2010/prov/2010_52.tif.
File data/input/GHSL Data/2010/prov/2010_81.tif uploaded to Degree of Urbanisation/GHSL data/2010/prov/2010_81.tif.
File data/input/GHSL Data/2010/prov/2010_75.tif uploaded to Degree of Urbanisation/GHSL data/2010/prov/2010_75.tif.
File data/input/GHSL Data/2010/prov/2010_18.tif uploaded to Degree of Urbanisation/GHSL data/2010/prov/2010_18.tif.
File data/input/GHSL Data/2010/prov/2010_36.tif uploaded to Degree of Urbanisation/GHSL data/2010/prov/2010_36.tif.
File data/input/GHSL Data/2010/prov/2010_13.tif uploaded to Degree of Urbanisation/GHSL data/2010/prov/2010_13.tif.
File data/input/GHSL Data/2010/prov/2010_73.tif uploaded to Degree of Ur